| ISS Tracker | International Space Station Tracker  |
| ----------- | ----------- |
| Date | Feb 2023 |
| Base API url | http://api.open-notify.org/ |
| Base API url | http://api.open-notify.org/ |
| Base API url | http://api.open-notify.org/ |
| Base API url | http://api.open-notify.org/ |

In [8]:
# libraries
import requests
from geopy.geocoders import Nominatim

# config file(s) import

In [2]:
"""Get the current location of the ISS"""
# send a GET request to the ISS API endpoint
response = requests.get("http://api.open-notify.org/iss-now.json")
data = response.json()

# print the API parameters
print(data)

{'message': 'success', 'timestamp': 1676332027, 'iss_position': {'longitude': '-118.3038', 'latitude': '19.7440'}}


In [6]:
# extract the latitude and longitude for the current location of the ISS
iss_latitude = data['iss_position']['latitude']
iss_longitude = data['iss_position']['longitude']

# print the current latitude and longitude for the ISS
print("Current ISS Location:")
print(f"Latitude: {iss_latitude}")
print(f"Longitude: {iss_longitude}")

# create a geolocator object


Current ISS Location:
Latitude: 19.7440
Longitude: -118.3038


In [14]:
iss_latitude

'19.7440'

In [15]:
proximity_latitude = iss_latitude
proximity_longitude = iss_longitude




geolocator = Nominatim(user_agent="my-app")
iss_proximity_location = geolocator.reverse(f'{iss_latitude}, {iss_longitude}')
print(iss_proximity_location)

# # extract address components from the ISS location
# address = iss_proximity_location.raw["address"]
# country = address.get("country", "")
# ocean = address.get("ocean", "")
# sea = address.get("sea", "")

# if country:
#     print(f"Country: {country}")
# if ocean:
#     print(f"Ocean: {ocean}")
# if sea:
#     print(f"Sea: {sea}")
# else:
#     print("Location not found.")



None


In [16]:
from geopy.geocoders import Nominatim

# Create a geolocator object
geolocator = Nominatim(user_agent="my-app")

# Define the latitude and longitude
latitude = 40.7128
longitude = -74.0060

# Use the geolocator to reverse geocode the location
location = geolocator.reverse(f"{latitude}, {longitude}")

if location is not None:
    # Extract the address components from the location
    address = location.raw["address"]
    country = address.get("country", "")
    ocean = address.get("ocean", "")
    sea = address.get("sea", "")

    # Print the results
    if country:
        print(f"Country: {country}")
    if ocean:
        print(f"Ocean: {ocean}")
    if sea:
        print(f"Sea: {sea}")
else:
    print("Location not found.")


Country: United States


In [ ]:
type(latitude)